1. Con respecto a la librería multiprocessing, utilizar la función starmap para crear los procesos que se van a ejecutar en paralelo. La función starmap tiene la ventaja de que puede crear procesos pasándole diversos parámetros, por lo que no será necesario realizar diversas copias del array original al que queremos realizar la operación de reducción de sus elementos. Para ello, lo primero es modificar la función de la reducción def reduc_operation(A) para que te permita pasarle, además del array a sumar, otros 2 parámetros adicionales, el de inicio y el de fin de la operación de reducción. Por tanto, dicha función debe quedar así: def sum_multiprocessing(A, ini, fin). A continuación, debes llamar a la nueva función sum_multiprocessing creando solo un proceso que sume todo el array (valores [0, value]), creando 2 procesos que sume la mitad del array (valores [(0, int(value/2)), (int(value/2), value)]), y creando 4 procesos que sumen cada uno la cuarta parte del array (valores [(0, int(value/4)), (int(value/4), int(value/2)), (int(value/2), int(3*value/4)),(int(3*value/4), value)]).

In [24]:
import numpy as np
from multiprocessing import Pool

# Modificamos la función de reducción para aceptar un inicio y fin
def sum_multiprocessing(A, ini, fin):
    """Sum the elements of Array A in the range [ini, fin)."""
    return np.sum(A[ini:fin])

# Secuencial
value = 5 * 10**4  # Tamaño del array
X = np.random.rand(value)  # Array con valores aleatorios

# Tiempo secuencial
print("Tiempo para la operación de reducción secuencial:")
tiempo = %timeit -r 2 -o -q np.sum(X)
print(f"Time taken by reduction operation using numpy.sum(): {tiempo}")
print(f"Resultado de la suma secuencial: {np.sum(X)}\n")


# Divisiones del array para multiprocessing
def dividir_array(value, num_procesos):
    """Divide el array en `num_procesos` subarrays según el esquema definido."""
    if num_procesos == 1:
        return [(0, value)]  # Una sola partición [0, value]
    elif num_procesos == 2:
        mitad = int(value / 2)
        return [(0, mitad), (mitad, value)]  # Dos particiones [0, mitad] y [mitad, value]
    elif num_procesos == 4:
        cuarto = int(value / 4)
        mitad = int(value / 2)
        tres_cuartos = int(3 * value / 4)
        return [(0, cuarto), (cuarto, mitad), (mitad, tres_cuartos), (tres_cuartos, value)]  # Cuatro particiones
    else:
        raise ValueError("Número de procesos no soportado. Use 1, 2 o 4.")

# Utilizando multiprocessing con 1, 2 y 4 procesos
for num_procesos in [1, 2, 4]:
    print(f"\nTiempo para la operación de reducción con {num_procesos} procesos:")

    # Dividimos el array según el número de procesos
    rangos = dividir_array(value, num_procesos)

    # Definimos la función que ejecutará multiprocessing
    def run_parallel_reduction():
        with Pool(processes=num_procesos) as pool:
            # Usamos starmap para aplicar sum_multiprocessing a cada rango
            resultados = pool.starmap(sum_multiprocessing, [(X, ini, fin) for (ini, fin) in rangos])
        return sum(resultados)

    # Medimos el tiempo usando %timeit
    tiempo = %timeit -r 2 -o -q run_parallel_reduction()
    resultado = run_parallel_reduction()

    print(f"Time taken by reduction operation using {num_procesos} processes: {tiempo}")
    print(f"Resultado de la suma con {num_procesos} procesos: {resultado}")

Tiempo para la operación de reducción secuencial:
Time taken by reduction operation using numpy.sum(): 14.2 µs ± 3.42 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Resultado de la suma secuencial: 25030.963157980892


Tiempo para la operación de reducción con 1 procesos:
Time taken by reduction operation using 1 processes: 13.4 ms ± 57.2 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Resultado de la suma con 1 procesos: 25030.963157980892

Tiempo para la operación de reducción con 2 procesos:
Time taken by reduction operation using 2 processes: 21.3 ms ± 62.9 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Resultado de la suma con 2 procesos: 25030.963157980896

Tiempo para la operación de reducción con 4 procesos:
Time taken by reduction operation using 4 processes: 38.5 ms ± 64.5 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Resultado de la suma con 4 procesos: 25030.963157980896


2. Con respecto a la librería cupy, en el apartado 3.3 hemos creado el array directamente en la GPU. Lo normal es que el array esté creado en la CPU, y que se copie dicho array a la GPU, se calcule la suma de los elementos, y el valor obtenido se devuelva a la CPU. De esta forma, el tiempo que se tarda en ejecutar la suma en la GPU es más real.

In [21]:
import cupy as cp
import numpy as np
import time

# Crear el array en la CPU usando numpy
value = 5 * 10**4  # Tamaño del array
X_cpu = np.random.rand(value)  # Array en la CPU con valores aleatorios

# Copiar el array a la GPU usando cupy
X_gpu = cp.asarray(X_cpu)

# Medir el tiempo de la operación de reducción en la GPU
start_time = time.time()

# Realizar la suma en la GPU usando cupy
result_cupy_sum = cp.sum(X_gpu)

end_time = time.time()

# Medir el tiempo de ejecución
time_taken = (end_time - start_time) * 1e6  # Convertir a microsegundos

print(f"Time taken by reduction operation using cupy.sum(): {time_taken:.2f} µs")
print(f"Result of the sum using cupy.sum(): {result_cupy_sum}\n")

# Copiar el resultado de vuelta a la CPU
result_cpu = cp.asnumpy(result_cupy_sum)

# Verificar que el resultado es el mismo que en la CPU
print(f"Result in CPU after transferring from GPU: {result_cpu}")

Time taken by reduction operation using cupy.sum(): 140.19 µs
Result of the sum using cupy.sum(): 24966.02640021457

Result in CPU after transferring from GPU: 24966.02640021457


3. Con respecto a la librería numba, numba ofrece un decorador @reduce para convertir una operación binaria simple en un núcleo de reducción. Prueba a utilizar este decorador para calcular la reducción del vector.

In [23]:
import numba
import numpy as np
import time

# Crear el array en la CPU usando numpy
value = 5 * 10**4  # Tamaño del array
X_cpu = np.random.rand(value)  # Array en la CPU con valores aleatorios

# Definir una función de reducción paralela usando numba.njit
@numba.njit(parallel=True)
def parallel_reduce(array):
    total = 0.0
    for i in numba.prange(array.size):  # Paralelizar el bucle con prange
        total += array[i]
    return total

# Medir el tiempo de ejecución de la reducción en la CPU con numba
start_time = time.time()

# Realizar la reducción
result_numba_sum = parallel_reduce(X_cpu)

end_time = time.time()

# Medir el tiempo de ejecución
time_taken = end_time - start_time

print(f"Time taken by reduction operation using Numba parallel_reduce: {time_taken:.6f} seconds")
print(f"Result of the sum using Numba parallel_reduce: {result_numba_sum}\n")

Time taken by reduction operation using Numba parallel_reduce: 0.191020 seconds
Result of the sum using Numba parallel_reduce: 25141.10815283626



El trabajo extra sería implementar estos pasos en Python y realizar las nuevas mediciones de tiempo. Pon el nombre que te parezca a este nuevo notebook, donde en la última celda del notebook muestres los resultados obtenidos en una de las colas del clúster de tu elección y comentes el porqué de dichos resultados.

En este ejercicio, se compararon tres enfoques diferentes para optimizar la operación de reducción en un array grande: multiprocessing, CuPy para la GPU, y Numba para la CPU. Cada uno de estos métodos tiene sus características, y los resultados obtenidos muestran cómo varían los tiempos de ejecución y los resultados finales.

La mejor opción en términos de rendimiento fue CuPy para la reducción en la GPU, ya que ofreció un tiempo de ejecución considerablemente más rápido que los métodos de multiprocessing y Numba. El método de multiprocessing, aunque útil para dividir la carga de trabajo en la CPU, mostró tiempos de ejecución más altos debido a la sobrecarga asociada con la gestión y sincronización de procesos. Por otro lado, Numba en la CPU, aunque eficiente y más rápido que multiprocessing, no pudo igualar el desempeño de CuPy, ya que el paralelismo en la CPU está limitado por el número de núcleos disponibles.

En resumen, para operaciones de reducción de grandes arrays, CuPy resultó ser la opción más eficiente en términos de tiempo de ejecución, especialmente cuando se dispone de una GPU.